In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [ ]:
# Initialize the Chrome driver
driver = webdriver.Chrome()


In [ ]:
driver.get('https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')

In [ ]:
# Function to log in to Amazon
def login_to_amazon(phone_number, password):
    driver.get('https://www.amazon.in/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.in%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=inflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')
    
    # Wait for the phone number field to load
    phone_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ap_email'))
    )
    phone_field.send_keys(phone_number)
    phone_field.send_keys(Keys.RETURN)
    
    # Wait for the password field to load
    password_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ap_password'))
    )
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)
    
    # Wait for login to complete
    time.sleep(5)  # Adjust sleep time as needed

In [ ]:

# Replace with your Amazon credentials
PHONE_NUMBER = '7820983685'  # Replace with your Amazon phone number
PASSWORD = 'nagesh09876'  # Replace with your Amazon password


In [ ]:
# calling the function:
login_to_amazon(PHONE_NUMBER, PASSWORD)

In [ ]:
# Navigating to the product reviews page recent revives:
driver.get("https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX3TW6X/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1")


# NOW SCRAPPING REVIVES

In [ ]:
# an empty DataFrame to store all reviews
all_reviews_df = pd.DataFrame(columns=['Review'])

In [ ]:
# Initialize an empty DataFrame to store all reviews
all_reviews_df = pd.DataFrame(columns=[
    'Reviewer Name', 'Stars', 'Review Title', 'Review', 'Review Date', 
    'Verified Purchase', 'Helpful Votes'
])

# Loop through all pages of reviews
while True:
    # Wait for the page to load completely
    time.sleep(5)  # Adjust sleep time as needed

    # Parse the page source using BeautifulSoup
    html_data = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract reviews
    reviews = html_data.find_all('div', {'data-cel-widget': True})  # Find all review containers
    
    # Store in a list
    review_data = []
    
    for review in reviews:
        # Extract reviewer name
        reviewer_name_tag = review.find('span', {'class': 'a-profile-name'})
        reviewer_name = reviewer_name_tag.text.strip() if reviewer_name_tag else 'Unknown'
        
        # Extract star rating
        star_rating_tag = review.find('span', {'class': 'a-icon-alt'})
        star_rating = star_rating_tag.text.strip() if star_rating_tag else 'Unknown'
        
        # Extract review title
        review_title_tag = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title_tag.text.strip() if review_title_tag else 'Unknown'
        
        # Extract review text
        review_text_tag = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text_tag.text.strip() if review_text_tag else 'Unknown'
        
        # Extract review date
        review_date_tag = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date_tag.text.strip() if review_date_tag else 'Unknown'
        
        # Extract verified purchase
        verified_purchase_tag = review.find('span', {'data-hook': 'avp-badge'})
        verified_purchase = verified_purchase_tag.text.strip() if verified_purchase_tag else 'No'
        
        # Extract helpful votes
        helpful_votes_tag = review.find('span', {'data-hook': 'helpful-vote-statement'})
        helpful_votes = helpful_votes_tag.text.strip() if helpful_votes_tag else '0'
        
        # Append to the list
        review_data.append([
            reviewer_name, star_rating, review_title, review_text, review_date,
            verified_purchase, helpful_votes
        ])
    
    # Convert to DataFrame
    page_reviews_df = pd.DataFrame(review_data, columns=[
        'Reviewer Name', 'Stars', 'Review Title', 'Review', 'Review Date', 
        'Verified Purchase', 'Helpful Votes'
    ])
    
    # Append to the main DataFrame
    all_reviews_df = pd.concat([all_reviews_df, page_reviews_df], ignore_index=True)
    
    # Check if there is a "Next Page" button
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, 'a-last')
        if 'a-disabled' in next_page_button.get_attribute('class'):
            break  # Exit the loop if there is no next page
        next_page_button.click()
    except Exception as e:
        print("No more pages or error navigating to the next page:", e)
        break

# Close the driver
#driver.quit()

# Display the final DataFrame
print(all_reviews_df)

# Save the DataFrame to a CSV file
all_reviews_df.to_csv('amazon_reviews_recent.csv', index=False)

    Reviewer Name                   Stars Review Title   Review Review Date  \
0         Unknown                 Unknown      Unknown  Unknown     Unknown   
1         Unknown                 Unknown      Unknown  Unknown     Unknown   
2         Unknown      4.5 out of 5 stars      Unknown  Unknown     Unknown   
3         Unknown      4.5 out of 5 stars      Unknown  Unknown     Unknown   
4         Unknown                 Unknown      Unknown  Unknown     Unknown   
..            ...                     ...          ...      ...         ...   
243       Unknown                 Unknown      Unknown  Unknown     Unknown   
244       Unknown                 Unknown      Unknown  Unknown     Unknown   
245       Unknown                 Unknown      Unknown  Unknown     Unknown   
246       Unknown  Previous set of slides      Unknown  Unknown     Unknown   
247       Unknown  Previous set of slides      Unknown  Unknown     Unknown   

    Verified Purchase Helpful Votes  
0            

In [ ]:
# Navigating to the product reviews page
driver.get('https://www.amazon.in/Apple-iPhone-15-128-GB/product-reviews/B0CHX3TW6X/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

In [ ]:
# an empty DataFrame to store all reviews
all_reviews_df = pd.DataFrame(columns=['Review'])

In [ ]:
# Initialize an empty DataFrame to store all reviews
all_reviews_df = pd.DataFrame(columns=[
    'Reviewer Name', 'Stars', 'Review Title', 'Review', 'Review Date', 
    'Verified Purchase', 'Helpful Votes'
])

# Loop through all pages of reviews
while True:
    # Wait for the page to load completely
    time.sleep(5)  # Adjust sleep time as needed

    # Parse the page source using BeautifulSoup
    html_data = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract reviews
    reviews = html_data.find_all('div', {'data-cel-widget': True})  # Find all review containers
    
    # Store in a list
    review_data = []
    
    for review in reviews:
        # Extract reviewer name
        reviewer_name_tag = review.find('span', {'class': 'a-profile-name'})
        reviewer_name = reviewer_name_tag.text.strip() if reviewer_name_tag else 'Unknown'
        
        # Extract star rating
        star_rating_tag = review.find('span', {'class': 'a-icon-alt'})
        star_rating = star_rating_tag.text.strip() if star_rating_tag else 'Unknown'
        
        # Extract review title
        review_title_tag = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title_tag.text.strip() if review_title_tag else 'Unknown'
        
        # Extract review text
        review_text_tag = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text_tag.text.strip() if review_text_tag else 'Unknown'
        
        # Extract review date
        review_date_tag = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date_tag.text.strip() if review_date_tag else 'Unknown'
        
        # Extract verified purchase
        verified_purchase_tag = review.find('span', {'data-hook': 'avp-badge'})
        verified_purchase = verified_purchase_tag.text.strip() if verified_purchase_tag else 'No'
        
        # Extract helpful votes
        helpful_votes_tag = review.find('span', {'data-hook': 'helpful-vote-statement'})
        helpful_votes = helpful_votes_tag.text.strip() if helpful_votes_tag else '0'
        
        # Append to the list
        review_data.append([
            reviewer_name, star_rating, review_title, review_text, review_date,
            verified_purchase, helpful_votes
        ])
    
    # Convert to DataFrame
    page_reviews_df = pd.DataFrame(review_data, columns=[
        'Reviewer Name', 'Stars', 'Review Title', 'Review', 'Review Date', 
        'Verified Purchase', 'Helpful Votes'
    ])
    
    # Append to the main DataFrame
    all_reviews_df = pd.concat([all_reviews_df, page_reviews_df], ignore_index=True)
    
    # Check if there is a "Next Page" button
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, 'a-last')
        if 'a-disabled' in next_page_button.get_attribute('class'):
            break  # Exit the loop if there is no next page
        next_page_button.click()
    except Exception as e:
        print("No more pages or error navigating to the next page:", e)
        break

# Close the driver
#driver.quit()

# Display the final DataFrame
print(all_reviews_df)

# Save the DataFrame to a CSV file
all_reviews_df.to_csv('amazon_reviews_top.csv', index=False)

In [ ]:
# top revives dataframe:
df1=pd.read_csv(r"C:\Users\Nagesh Agrawal\OneDrive\Desktop\sentiment project\amazon_reviews_top.csv")
df1

,Reviewer Name,Stars,Review Title,Review,Review Date,Verified Purchase,Helpful Votes
0,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
1,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
2,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
3,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
4,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
...,...,...,...,...,...,...,...
243,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
244,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
245,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
246,Unknown,Previous set of slides,Unknown,Unknown,Unknown,No,0


In [ ]:
# recent revives dataframe:
df2=pd.read_csv(r"C:\Users\Nagesh Agrawal\OneDrive\Desktop\sentiment project\amazon_reviews_recent.csv")
df2

,Reviewer Name,Stars,Review Title,Review,Review Date,Verified Purchase,Helpful Votes
0,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
1,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
2,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
3,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
4,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
...,...,...,...,...,...,...,...
243,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
244,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
245,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
246,Unknown,Previous set of slides,Unknown,Unknown,Unknown,No,0


In [ ]:
df3 = pd.concat([df1, df2], ignore_index=True)


In [ ]:
df3

,Reviewer Name,Stars,Review Title,Review,Review Date,Verified Purchase,Helpful Votes
0,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
1,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
2,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
3,Unknown,4.5 out of 5 stars,Unknown,Unknown,Unknown,No,0
4,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
...,...,...,...,...,...,...,...
491,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
492,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
493,Unknown,Unknown,Unknown,Unknown,Unknown,No,0
494,Unknown,Previous set of slides,Unknown,Unknown,Unknown,No,0


In [ ]:
# Save the DataFrame to a CSV file
df3.to_csv('amazon_reviews_all.csv', index=False)